In [1]:
import os
import os.path as osp
import pickle
import time
from typing import NamedTuple
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from mujoco_py import const
import wrappers
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import torch
import torch.nn.functional as F
import copy

from garage import EpisodeBatch
import torch
from torch import nn
from torchsummary import summary_string

# import dowel
# from dowel import logger, tabular
import gym

from garage.torch.modules import MLPModule
torch.set_num_threads(4)
from buffers import PreferenceCollector, LabelAnnealer, SyntheticPreferenceCollector
from reward_predictors import PrefMLP

import wandb

In [2]:
def get_episodes(exps, epochs):  
    episodes = []    
    for exp in exps:
        for epoch in epochs:
            filename = osp.join('./experiment', exp, 'episode_logs', f'episode_{str(epoch)}.pkl')        
            with open(filename, 'rb') as f:
                episodes.append(pickle.load(f))
    episodes = EpisodeBatch.concatenate(*episodes)
    return episodes

In [3]:
with open(osp.join('./experiment', 'DIVERSITY_ALPHA_05_Mon_Feb__1_09:36:50_2021', 'env.pkl'), 'rb') as f:
    env = pickle.load(f)
    
with open(osp.join('./experiment', 'DIVERSITY_RUN_0_Thu_Feb__4_13:04:03_2021', 'env.pkl'), 'rb') as f:
    reversed_env = pickle.load(f)

eps = get_episodes(['DIVERSITY_ALPHA_05_Mon_Feb__1_09:36:50_2021'], [200])
    
# val_eps = get_episodes(
#     exps=['DIVERSITY_RUN_0_Thu_Feb__4_07:46:02_2021', 'DIVERSITY_RUN_1_Thu_Feb__4_07:46:05_2021',
#     'DIVERSITY_RUN_2_Thu_Feb__4_07:46:09_2021', 'DIVERSITY_RUN_3_Thu_Feb__4_07:46:11_2021'], 
#     epochs=[200, 400])

# reversed_eps = get_episodes(
#     exps=['DIVERSITY_RUN_0_Thu_Feb__4_13:04:03_2021', 'DIVERSITY_RUN_1_Thu_Feb__4_13:04:06_2021',
#     'DIVERSITY_RUN_2_Thu_Feb__4_13:04:13_2021', 'DIVERSITY_RUN_3_Thu_Feb__4_13:04:17_2021'], 
#     epochs=[200])

# space = env.spec.observation_space
# skip = 40
# observations = np.array([space.flatten_with_keys(x, keys=['state']) for x in eps.observations[::skip]])
# rewards = eps.env_infos['gt_reward'][::skip]
# assert len(observations) == len(rewards)

/opt/conda/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
label_anealler = LabelAnnealer(100, 200, 0)
pref_collector = SyntheticPreferenceCollector(env.spec, label_anealler, max_capacity=10000)

In [5]:
pref_collector.collect(eps.to_list())

In [6]:
pref_collector.sample_comparisons(101)

In [7]:
pref_collector.label_unlabeled_comparisons()

In [8]:
reward_predictor = PrefMLP(
    env_spec=env.spec,
    preference_collector=pref_collector,
)

In [9]:
reward_predictor._train_once()

In [10]:
itr = 1
paths = eps.to_list()[0:10]

In [11]:
paths = reward_predictor.predict_rewards(itr, paths)

In [22]:
from garage.torch import np_to_torch

inp = np_to_torch(env.spec.observation_space.flatten_n(eps.observations[0:4]))

In [30]:
reward_predictor(inp.reshape(1,1,4,96)).ndim

4

In [14]:
eval('PreferenceCollector')

buffers.preference_collector.PreferenceCollector

In [17]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--name', type=str)
args = parser.parse_args(['--name', 'bob'])

In [19]:
args.name

'bob'

In [26]:
import envs.custom_safety_envs
env_ = gym.make('Safexp-PointIRLGoalThree-v0')

In [29]:
env_.spec.max_episode_steps

In [30]:
from garage.envs import GymEnv
GymEnv(env_, max_episode_length=500).spec.max_episode_length

500